In [4]:
# 1. Importar la biblioteca necesaria
from ortools.sat.python import cp_model

def resolver_job_shop():
    """Resuelve nuestro problema de Job Shop 3x3."""
    # 2. Definir los datos del problema
    # Formato: [(maquina, tiempo_proceso), ...] para cada trabajo
    jobs_data = [
        [(1, 5), (2, 3), (0, 2)],  # Trabajo 1: O1,3(M1,5), O1,2(M3,3), O1,1(M2,2)
                                   # Nota: El orden aquí es el que definimos,
                                   # pero la precedencia la forzaremos en el modelo.
                                   # Para OR-Tools, M1=0, M2=1, M3=2
        [(0, 3), (1, 6), (2, 4)],  # Trabajo 2: O2,1(M1,3), O2,2(M2,6), O2,3(M3,4)
        [(1, 2), (0, 2), (2, 4)]   # Trabajo 3: O3,1(M2,2), O3,2(M1,2), O3,3(M3,4)
    ]
    
    # Re-mapeo para coincidir con nuestra conversacion (M1=1, M2=2, M3=3)
    # y nuestro orden de operaciones O_i,1 -> O_i,2 -> O_i,3
    # Formato: (maquina_id, tiempo_proceso)
    jobs_data_correcto = [
        [(1, 2), (2, 3), (0, 5)], # J1: O1,1(M2,2), O1,2(M3,3), O1,3(M1,5)
        [(0, 3), (1, 6), (2, 4)], # J2: O2,1(M1,3), O2,2(M2,6), O2,3(M3,4)
        [(1, 2), (0, 2), (2, 4)]  # J3: O3,1(M2,2), O3,2(M1,2), O3,3(M3,4)
    ]
    
    num_jobs = len(jobs_data_correcto)
    num_machines = 3

    # 3. Crear el modelo
    model = cp_model.CpModel()

    # --- Creación de Variables ---
    # Diccionario para guardar todas las tareas (operaciones)
    all_tasks = {}
    # Horizonte de tiempo máximo posible (suma de todos los tiempos)
    horizon = sum(task[1] for job in jobs_data_correcto for task in job)

    for job_id, job in enumerate(jobs_data_correcto):
        for task_id, task in enumerate(job):
            machine, duration = task
            suffix = f'_{job_id}_{task_id}'
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            # Un "IntervalVar" es el objeto clave que representa una tarea
            interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)
            all_tasks[(job_id, task_id)] = interval_var

    # --- Definición de Restricciones ---
    # a) Restricción de no superposición en las máquinas
    for machine_id in range(num_machines):
        intervals_on_machine = []
        for job_id, job in enumerate(jobs_data_correcto):
            for task_id, task in enumerate(job):
                if task[0] == machine_id:
                    intervals_on_machine.append(all_tasks[(job_id, task_id)])
        # Esta línea le dice al modelo que los intervalos en esta lista no pueden superponerse en el tiempo
        model.AddNoOverlap(intervals_on_machine)

    # b) Restricción de PRECEDENCIA dentro de un mismo trabajo
    for job_id, job in enumerate(jobs_data_correcto):
        for task_id in range(len(job) - 1):
            op_actual = all_tasks[(job_id, task_id)]
            op_siguiente = all_tasks[(job_id, task_id + 1)]
            # El inicio de la siguiente debe ser >= al fin de la actual
            model.Add(op_siguiente.StartExpr() >= op_actual.EndExpr())

    # --- Definición del Objetivo ---
    # El objetivo es minimizar el makespan (el tiempo de finalización más tardío)
    makespan = model.NewIntVar(0, horizon, 'makespan')
    
    last_ops = [all_tasks[(job_id, len(job) - 1)] for job_id, job in enumerate(jobs_data_correcto)]
    model.AddMaxEquality(makespan, [op.EndExpr() for op in last_ops])
    
    model.Minimize(makespan)

    # 4. Resolver el modelo
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # 5. Imprimir los resultados
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"Solución encontrada con Makespan Óptimo: {solver.ObjectiveValue()}\n")
        
        # Crear un resumen de la planificación
        schedule = []
        for job_id, job in enumerate(jobs_data_correcto):
            for task_id, task in enumerate(job):
                machine, _ = task
                start_time = solver.Value(all_tasks[(job_id, task_id)].StartExpr())
                end_time = solver.Value(all_tasks[(job_id, task_id)].EndExpr())
                schedule.append({
                    "Trabajo": f"J{job_id+1}",
                    "Operación": f"O_{job_id+1},{task_id+1}",
                    "Máquina": f"M{machine+1}",
                    "Inicio": start_time,
                    "Fin": end_time
                })
        
        # Imprimir la tabla de resultados
        # Ordenar por máquina y luego por tiempo de inicio para ver la secuencia
        schedule.sort(key=lambda x: (x["Máquina"], x["Inicio"]))
        
        print("Plan de Producción Detallado:")
        for task in schedule:
            print(f"  {task['Trabajo']}-{task['Operación']}: Máquina {task['Máquina']} | Inicio: {task['Inicio']} | Fin: {task['Fin']}")
            
    else:
        print("No se encontró una solución óptima.")

# Ejecutar la función
resolver_job_shop()

Solución encontrada con Makespan Óptimo: 14.0

Plan de Producción Detallado:
  J2-O_2,1: Máquina M1 | Inicio: 0 | Fin: 3
  J3-O_3,2: Máquina M1 | Inicio: 4 | Fin: 6
  J1-O_1,3: Máquina M1 | Inicio: 6 | Fin: 11
  J1-O_1,1: Máquina M2 | Inicio: 0 | Fin: 2
  J3-O_3,1: Máquina M2 | Inicio: 2 | Fin: 4
  J2-O_2,2: Máquina M2 | Inicio: 4 | Fin: 10
  J1-O_1,2: Máquina M3 | Inicio: 2 | Fin: 5
  J3-O_3,3: Máquina M3 | Inicio: 6 | Fin: 10
  J2-O_2,3: Máquina M3 | Inicio: 10 | Fin: 14
